In [19]:
#Import Libraries
import random
import os
import pandas as pd

from pathlib import Path

import spacy
from spacy.util import minibatch, compounding

In [20]:
#Variables
srcPath=r'C:\\NLP\\'
inputDir=r'Input'
outputDir=r'Output'
modelDir=r'Model'

#Directory Path
trainFilesDir=r'TrainFiles'

#Excel document holding degree names
trainFileDegree=r'Degree.xlsx'

#Spacy model to use
spacyModel=r'en_core_web_lg'

In [21]:
#Initialize lists
train_data=[]
entities=[] 

In [22]:
#Check if Folder exists else Create
#Input Folder
if not os.path.exists(srcPath + inputDir):
    os.makedirs(srcPath + inputDir)

#Output Folder
if not os.path.exists(srcPath + outputDir):
    os.makedirs(srcPath + outputDir)
    
#Model Folder
if not os.path.exists(srcPath + modelDir):
    os.makedirs(srcPath + modelDir)
    
#TrainFiles Folder
if not os.path.exists(srcPath + trainFilesDir):
    os.makedirs(srcPath + trainFilesDir)

In [23]:
#Load Degree List from  Xlsx  file
degreeFilePath = srcPath + trainFilesDir + '\\' + trainFileDegree
df = pd.read_excel(degreeFilePath, sheet_name=0, header=0)
degreeList = df['Degree'].tolist()

In [24]:
#Load Degree Entities for training.
degreeListFormat1=degreeList[0:20]
degreeListFormat2=degreeList[21:40]
degreeListFormat3=degreeList[41:60]
degreeListFormat4=degreeList[61:80]


for  item in degreeListFormat1:
    a='completed '+ item.rstrip()
    entities.append(( 10, len(a) ,'ORG'))
    train_data.append((a, {"entities" : entities}))
    entities=[] 
    
    
for item in degreeListFormat2:    
    b= item.rstrip() + ' from Govt college'
    entities.append((0, len(item) ,'ORG'))
    train_data.append((b, {"entities" : entities}))
    entities=[]
    

for item in degreeListFormat3:    
    c= 'Education: ' + item.rstrip()
    entities.append((11, len(c) ,'ORG'))
    train_data.append((c, {"entities" : entities}))
    entities=[]
    
    
for item in degreeListFormat4:    
    d= 'Under Graduate: ' + item.rstrip()
    entities.append((16, len(d) ,'ORG'))
    train_data.append((d, {"entities" : entities}))
    entities=[]

In [25]:
#Define  model training function. This is Spacy Code
def train_Model(model, output_dir, n_iter=100):
   #Load the model, set up the pipeline and train the entity recognizer.
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")

        
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe('ner')

        
    # add labels
    for _, annotations in train_data:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

            
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(train_data)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(train_data, size=compounding(4., 32., 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)

            
    #save model to output directory
        outputDir =output_dir
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

In [26]:
#Train Model with new data
outputPath= srcPath + modelDir
train_Model(spacyModel,outputPath )

Loaded model 'en_core_web_lg'
Saved model to C:\\NLP\\Model


In [27]:
#Load the saved model
nlp2 = spacy.load(r'C:\NLP\Model')

In [28]:
#Test the model
TestDegrees = [
    ('I am a  Bachelor of Arts in Sanskrit'),
    ('I have completed Advanced diploma in Dredging technology'),
    ('I posses a degree in  Bachelor of Arts Philosophy from Govt college'),
    ('Certified in Bachelor of Ayurvedic Medicine & Surgery '),
    ('undergrauate in Bachelor of Economics')
]

for deg in TestDegrees:
            docDeg = nlp2(deg)
            print('Entities', [(ent.text, ent.label_) for ent in docDeg.ents])

Entities [('Bachelor of Arts in Sanskrit', 'ORG')]
Entities [('Advanced diploma in Dredging technology', 'ORG')]
Entities [(' Bachelor of Arts Philosophy', 'ORG')]
Entities [('Certified in Bachelor of Ayurvedic Medicine & Surgery', 'ORG')]
Entities [('Bachelor of Economics', 'ORG')]
